
```bash
# 1. Download the llamafile-ized model
wget https://huggingface.co/jartine/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile

# 2. Make executable 
chmod +x TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile

# 3. Run in server mode
./TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile --server --nobrowser --embedding --port 8080
```

In [8]:
# Install llama-index
!pip install llama-index-core
# Install llamafile integrations and SimpleWebPageReader
!pip install llama-index-embeddings-llamafile llama-index-llms-llamafile llama-index-readers-web
!pip install llama-index 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 948.0 kB/s eta 0:00:001m2.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.5/362.5 kB 1.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 1.5 MB/s eta 0:00:001.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.2/319.2 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:0101
  Attempting uninstall: openai
    Found existing installation: openai 1.23.3
    Uninstalling openai-1.23.3:
      Successfully uninstalled openai-1.23.3


In [9]:
# Configure LlamaIndex
from llama_index.core import Settings
from llama_index.embeddings.llamafile import LlamafileEmbedding
from llama_index.llms.llamafile import Llamafile
from llama_index.core.node_parser import SentenceSplitter

Settings.embed_model = LlamafileEmbedding(base_url="http://localhost:8080")

Settings.llm = Llamafile(
    base_url="http://localhost:8080",
    temperature=0,
    seed=0
)

Settings.transformations = [
    SentenceSplitter(
        chunk_size=256, 
        chunk_overlap=5
    )
]

In [10]:
# Load local data
from llama_index.core import SimpleDirectoryReader
local_doc_reader = SimpleDirectoryReader(input_dir='./data')
docs = local_doc_reader.load_data(show_progress=True)

Loading files: 100%|██████████| 2/2 [00:00<00:00, 151.09file/s]


In [11]:
# Add Wikipedia page
from llama_index.readers.web import SimpleWebPageReader
urls = [
    'https://pl.wikipedia.org/wiki/Nad_Niemnem',
]
web_reader = SimpleWebPageReader(html_to_text=True)
docs.extend(web_reader.load_data(urls))

In [12]:

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    docs,
    show_progress=True,
)

index.storage_context.persist(persist_dir="./storage")

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/94 [00:00<?, ?it/s]

In [14]:
query_engine = index.as_query_engine()
print(query_engine.query("What is revenue?"))

Revenue is the amount of money earned by a business or organization from its activities. It is the profit earned from selling goods or services, or from other sources of income.</s>
